In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [63]:
df = pd.read_csv('~/.jupyter/lab/workspaces/data_analysis/cars_dot_com/data/cars_by_popular_cities_cleaned.csv')

In [33]:
# Set display options to show all values
pd.set_option('display.max_rows', None)  # Set to None for unlimited rows
pd.set_option('display.max_columns', None)  # Set to None for unlimited columns

In [34]:
# Define lists of luxury and non-luxury brands
luxury_brands = ['Rolls-Royce', 'Porsche', 'Mercedes-Benz', 'BMW', 'Ferrari', 
                 'INFINITI', 'Audi', 'Lexus', 'Bentley', 'Maserati', 'Jaguar', 
                 'Volvo', 'Aston', 'Lamborghini', 'Tesla', 'Land', 'Rivian', 
                 'Alfa', 'Genesis', 'McLaren', 'Maybach', 'Bugatti', 'Karma', 
                 'AC', 'Polestar', 'Delorean', 'Lotus', 'Lucid', 'Hummer', 'Acura', 'Cadillac', 'Lincoln']

nonluxury_brands = ['Panoz', 'Chevrolet', 'Jeep', 'Volkswagen', 'Buick', 'Toyota', 
              'Nissan', 'Chrysler', 'Honda', 'MINI', 'Mazda', 'Hyundai', 'Ford', 
              'Mitsubishi', 'Subaru', 'Dodge', 'GMC', 'RAM', 'INEOS', 'Scion',
              'Kia', 'FIAT', 'Oldsmobile', 'Saturn', 'Plymouth', 'Pontiac',
               'Suzuki', 'Saab', 'Mercury', 'Isuzu']

In [35]:
df['year'] = df['year'].astype(int)

In [36]:
df['price'] = df['price'].fillna(0)  # Replace NaN with 0
df = df[df['price'] != 0]

In [37]:
# Drop duplicates
df.drop_duplicates(inplace=True)

In [38]:
# Update 'fuel_type' to 'Hybrid' if it contains 'Hybrid' or 'Electric and Gas Hybrid'
df['fuel_type'] = np.where(df['fuel_type'].str.contains('Hybrid|Electric and Gas Hybrid'), 'Hybrid', df['fuel_type'])

# Update 'fuel_type' to 'Electric' if it contains 'Electric'
df['fuel_type'] = np.where(df['fuel_type'].str.contains('Electric'), 'Electric', df['fuel_type'])

# Update 'fuel_type' to 'Gasoline' if it contains 'E85 Flex Fuel'
df['fuel_type'] = np.where(df['fuel_type'].str.contains('E85 Flex Fuel'), 'Gasoline', df['fuel_type'])

In [39]:
df['status'] = df['status'].map(lambda x: 'Used' if 'Certified' in x else x)
df['status'] = df.apply(lambda row: 'Used' if row['year'] <= 2022 else row['status'], axis=1)

In [40]:
# Calculate average price by brand and status where status is 'New' or 'Used'
avg_price_by_brand = df[df['status'].isin(['New', 'Used'])].groupby(['brand', 'status'])['price'].mean().astype(int)


# Function to replace prices <= 0 with average price
def replace_negative_prices(row):
    if row['price'] <= 0:
        if (row['brand'], row['status']) in avg_price_by_brand.index:
            return avg_price_by_brand.loc[(row['brand'], row['status'])]
        else:
            return row['price']  # If no average price found, return original price
    else:
        return row['price']

# Apply the function to update 'price' column
df['price'] = df.apply(replace_negative_prices, axis=1)

In [41]:
city_colors = {
    'atlanta': '#1f77b4',       # Blue
    'chicago': '#ff7f0e',       # Orange
    'columbus': '#2ca02c',      # Green
    'dallas': '#9467bd',        # Purple
    'denver': '#8c564b',        # Brown
    'houston': '#e377c2',       # Pink
    'los_angeles': '#7f7f7f',   # Gray
    'new_york': '#bcbd22',      # Olive
    'philadelphia': '#17becf',  # Cyan
    'phoenix': '#aec7e8',       # Light Blue
    'san_diego': '#ff9896',     # Salmon
    'seattle': '#ffcc00'        # Yellow
}


In [42]:
# Filter data for luxury and non-luxury brands
avg_lux_price_per_city = df[df['brand'].isin(luxury_brands)].copy()
avg_nonlux_price_per_city = df[df['brand'].isin(nonluxury_brands)].copy()

# Rename columns
avg_lux_price_per_city.rename(columns={
    'comfort': 'comfort_rating',
    'interior': 'interior_rating',
    'performance': 'performance_rating',
    'value': 'value_rating',
    'exterior': 'exterior_rating',
    'reliability': 'reliability_rating'
}, inplace=True)

avg_nonlux_price_per_city.rename(columns={
    'comfort': 'comfort_rating',
    'interior': 'interior_rating',
    'performance': 'performance_rating',
    'value': 'value_rating',
    'exterior': 'exterior_rating',
    'reliability': 'reliability_rating'
}, inplace=True)

# Filter by status
new_lux = avg_lux_price_per_city[avg_lux_price_per_city['status'] == 'New'].copy()
used_lux = avg_lux_price_per_city[avg_lux_price_per_city['status'] == 'Used'].copy()
new_nonlux = avg_nonlux_price_per_city[avg_nonlux_price_per_city['status'] == 'New'].copy()
used_nonlux = avg_nonlux_price_per_city[avg_nonlux_price_per_city['status'] == 'Used'].copy()

# Convert year to integer (if necessary)
new_lux['year'] = new_lux['year'].astype(int)
used_lux['year'] = used_lux['year'].astype(int)
new_nonlux['year'] = new_nonlux['year'].astype(int)
used_nonlux['year'] = used_nonlux['year'].astype(int)

# Define numerical columns
numerical_cols = ['year', 'mileage', 'seller_rating', 'consumer_rating', 'comfort_rating', 'interior_rating', 'performance_rating', 'value_rating', 'exterior_rating', 'reliability_rating']


## Numeric Distributions

#### New Luxury Distributions

In [43]:
# Sort reliability_rating column
new_lux_sorted = new_lux.sort_values(by='reliability_rating')

# Create subplots for each numerical column
fig = make_subplots(rows=4, cols=3, subplot_titles=numerical_cols)

# Add histograms to the subplots
for i, col in enumerate(numerical_cols):
    row = i // 3 + 1
    col_num = i % 3 + 1
    
    if col == 'reliability_rating':
        fig.add_trace(
            go.Histogram(x=new_lux_sorted[col], nbinsx=15, name=col),
            row=row, col=col_num
        )
    else:
        fig.add_trace(
            go.Histogram(x=new_lux[col], nbinsx=15, name=col),
            row=row, col=col_num
        )
    
    if col == 'year':
        fig.update_xaxes(tickvals=new_lux[col].unique(), tickformat='d', row=row, col=col_num)
    elif col in ['mileage', 'reliability_rating']:
        fig.update_xaxes(tickformat=',', row=row, col=col_num)
        fig.update_xaxes(tickangle=0, row=row, col=col_num)

# Update layout
fig.update_layout(
    title_text='Numeric New Luxury Distributions',
    height=1000,
    width=1200,
    showlegend=False
)

# Show the figure
# Show the figure without Plotly watermark
fig.show(config={'displaylogo': False})

## New Luxury Cars
- Year: Most cars are from 2024.
- Mileage: Most cars have very low mileage (below 400 miles).
- Seller Rating: Most sellers have high ratings (4.5 to 5 stars).
- Consumer Rating: Most cars have high consumer ratings (4.5 to 5 stars).
- Comfort Rating: Most cars have high comfort ratings (5 stars).
- Interior Rating: Most cars have high interior ratings (5 stars).
- Performance Rating: Most cars have high performance ratings (5 stars).
- Value Rating: Most cars have high value ratings (5 stars).
- Exterior Rating: Most cars have high exterior ratings (5 stars).
- Reliability Rating: Most cars have high reliability ratings (5 stars).

### Insights
- Age and Mileage: New luxury cars are from the latest model year (2024) and have very low mileage.
- Ratings: High ratings across various categories indicate overall satisfaction with new luxury cars.

#### Actionable Insight:
- Buyers: New luxury cars from 2024 with high ratings are a good choice for a reliable and high-quality purchase.
- Sellers: Emphasize the new model year and high ratings to attract buyers.

### Used Luxury Distributions

In [44]:
# Sort used_lux by reliability_rating column
used_lux_sorted = used_lux.sort_values(by='reliability_rating')

# Create subplots for each numerical column
fig = make_subplots(rows=4, cols=3, subplot_titles=numerical_cols)

# Add histograms to the subplots
for i, col in enumerate(numerical_cols):
    row = i // 3 + 1
    col_num = i % 3 + 1
    
    if col == 'reliability_rating':
        fig.add_trace(
            go.Histogram(x=used_lux_sorted[col], nbinsx=15, name=col),
            row=row, col=col_num
        )
    else:
        fig.add_trace(
            go.Histogram(x=used_lux[col], nbinsx=15, name=col),
            row=row, col=col_num
        )
    
    if col == 'year':
        # Determine tick values for year at intervals of 10 years
        min_year = used_lux[col].min()
        max_year = used_lux[col].max()
        tickvals = list(range(min_year - min_year % 10, max_year + (10 - max_year % 10), 10))
        fig.update_xaxes(tickvals=tickvals, tickformat='d', row=row, col=col_num)
    elif col in ['reliability_rating']:
        fig.update_xaxes(tickformat=',', row=row, col=col_num)
        fig.update_xaxes(tickangle=-60, row=row, col=col_num)

# Update layout
fig.update_layout(
    title_text='Numeric Used Luxury Distributions',
    height=1000,
    width=1200,
    showlegend=False
)

# Show the figure without Plotly watermark
fig.show(config={'displaylogo': False})

## Used Luxury Cars
- Year: Most cars are from 2010 onwards, with a peak around 2020.
- Mileage: Most cars have mileage below 100,000 miles.
- Seller Rating: Most sellers have high ratings (4 to 5 stars).
- Consumer Rating: Most cars have high consumer ratings (5 stars).
- Comfort Rating: Most cars have high comfort ratings (5 stars).
- Interior Rating: Most cars have high interior ratings (5 stars).
- Performance Rating: Most cars have high performance ratings (5 stars).
- Value Rating: Most cars have high value ratings (5 stars).
- Exterior Rating: Most cars have high exterior ratings (5 stars).
- Reliability Rating: Most cars have high reliability ratings (5 stars), but there is a notable number of cars with lower ratings.


### Insights
- Age and Mileage: Most used non-luxury cars are relatively new (post-2010) and have moderate mileage.
- Ratings: High ratings across various categories indicate overall satisfaction with used non-luxury cars.

#### Actionable Insight:
- Buyers: Look for used non-luxury cars from recent years with high ratings for a reliable purchase.
- Sellers: Highlight the high ratings and relatively low mileage of your cars to attract buyers.

## New Non-Luxury Distributions

In [45]:
# Sort new_nonlux by reliability_rating column
new_nonlux_sorted = new_nonlux.sort_values(by='reliability_rating')

# Create subplots for each numerical column
fig = make_subplots(rows=4, cols=3, subplot_titles=numerical_cols)

# Add histograms to the subplots
for i, col in enumerate(numerical_cols):
    row = i // 3 + 1
    col_num = i % 3 + 1
    
    if col == 'reliability_rating':
        fig.add_trace(
            go.Histogram(x=new_nonlux_sorted[col], nbinsx=15, name=col),
            row=row, col=col_num
        )
    else:
        fig.add_trace(
            go.Histogram(x=new_nonlux[col], nbinsx=15, name=col),
            row=row, col=col_num
        )
    
    if col == 'year':
        fig.update_xaxes(tickvals=new_nonlux[col].unique(), tickformat='d', row=row, col=col_num)
    elif col in ['mileage', 'reliability_rating']:
        fig.update_xaxes(tickformat=',', row=row, col=col_num)
        fig.update_xaxes(tickangle=-60, row=row, col=col_num)

# Update layout
fig.update_layout(
    title_text='Numeric New Non-Luxury Distributions',
    height=1000,
    width=1200,
    showlegend=False
)

# Show the figure without Plotly watermark
fig.show(config={'displaylogo': False})

## New Non-Luxury Cars
- Year: Most cars are from 2024.
- Mileage: Most cars have very low mileage (below 2,000 miles).
- Seller Rating: Most sellers have high ratings (4.5 to 5 stars).
- Consumer Rating: Most cars have high consumer ratings (4.5 to 5 stars).
- Comfort Rating: Most cars have high comfort ratings (5 stars).
- Interior Rating: Most cars have high interior ratings (5 stars).
- Performance Rating: Most cars have high performance ratings (5 stars).
- Value Rating: Most cars have high value ratings (5 stars).
- Exterior Rating: Most cars have high exterior ratings (5 stars).
- Reliability Rating: Most cars have high reliability ratings (5 stars).
#### Insights:
- Age and Mileage: New luxury cars are from the latest model year (2024) and have very low mileage.
- Ratings: High ratings across various categories indicate overall satisfaction with new luxury cars.
#### Actionable Insight:
- Buyers: New luxury cars from 2024 with high ratings are a good choice for a reliable and high-quality purchase.
- Sellers: Emphasize the new model year and high ratings to attract buyers.


## Used Non-Luxury Distributions

In [46]:
# Sort used_nonlux by reliability_rating column
used_nonlux_sorted = used_nonlux.sort_values(by='reliability_rating')

# Create subplots for each numerical column
fig = make_subplots(rows=4, cols=3, subplot_titles=numerical_cols)

# Add histograms to the subplots
for i, col in enumerate(numerical_cols):
    row = i // 3 + 1
    col_num = i % 3 + 1
    
    if col == 'reliability_rating':
        fig.add_trace(
            go.Histogram(x=used_nonlux_sorted[col], nbinsx=15, name=col),
            row=row, col=col_num
        )
    else:
        fig.add_trace(
            go.Histogram(x=used_nonlux[col], nbinsx=15, name=col),
            row=row, col=col_num
        )
    
    if col == 'year':
        # Determine tick values for year at intervals of 10 years
        min_year = used_nonlux[col].min()
        max_year = used_nonlux[col].max()
        tickvals = list(range(min_year - min_year % 10, max_year + (10 - max_year % 10), 10))
        fig.update_xaxes(tickvals=tickvals, tickformat='d', row=row, col=col_num)
    elif col in ['mileage', 'reliability_rating']:
        fig.update_xaxes(tickformat=',', row=row, col=col_num)
        fig.update_xaxes(tickangle=-60, row=row, col=col_num)

# Update layout
fig.update_layout(
    title_text='Numeric Used Non-Luxury Distributions',
    height=1000,
    width=1200,
    showlegend=False
)

# Show the figure without Plotly watermark
fig.show(config={'displaylogo': False})

## Used Non-Luxury Cars
- Year: Most cars are from 2010 onwards, with a peak around 2020.
- Mileage: Most cars have mileage below 100,000 miles.
- Seller Rating: Most sellers have high ratings (4 to 5 stars).
- Consumer Rating: Most cars have high consumer ratings (5 stars).
- Comfort Rating: Most cars have high comfort ratings (5 stars).
- Interior Rating: Most cars have high interior ratings (5 stars).
- Performance Rating: Most cars have high performance ratings (5 stars).
- Value Rating: Most cars have high value ratings (5 stars).
- Exterior Rating: Most cars have high exterior ratings (5 stars).
- Reliability Rating: Most cars have high reliability ratings (5 stars), but there is a notable number of cars with lower ratings.
#### Insights:
- Age and Mileage: Most used non-luxury cars are relatively new (post-2010) and have moderate mileage.
- Ratings: High ratings across various categories indicate overall satisfaction with used non-luxury cars.
#### Actionable Insight:
- Buyers: Look for used non-luxury cars from recent years with high ratings for a reliable purchase.
- Sellers: Highlight the high ratings and relatively low mileage of your cars to attract buyers.



### General Insights
- Age and Mileage: Both new luxury and non-luxury cars have very low mileage, indicating they are mostly brand new. Used luxury cars have moderate mileage, suggesting they have been driven for a few years, while used non-luxury cars have higher mileage, indicating extensive use.
- Ratings: Most cars, regardless of category, have high ratings across various attributes such as consumer satisfaction, comfort, interior, performance, value, exterior, and reliability.
#### Actionable Insights:
- For Buyers: Look for high ratings and low mileage for a reliable purchase. Consider the affordability and value offered by used cars despite higher mileage.
- For Sellers: Emphasize the exclusivity and high value of luxury brands, the low mileage of new cars, and the affordability of used cars to attract buyers.


## Total AVG Price Distribution per New and Used Luxury and Non-Luxury Brands

### AVG Price Distribution for New and Used Luxury Brands

In [47]:
# Create subplots with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, subplot_titles=('AVG New Luxury Price Distribution', 'AVG Used Luxury Price Distribution'))

# Plot AVG New Luxury
fig.add_trace(
    px.box(new_lux, x='price').data[0],
    row=1, col=1
)

# Plot AVG Used Luxury
fig.add_trace(
    px.box(used_lux, x='price').data[0],
    row=1, col=2
)

# Update layout for the entire figure
fig.update_layout(
    xaxis_title='Price',
    height=800,
    width=1150,
    title={
        #'text': 'Combined Luxury Price Distributions',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 18, 'family': 'bold'}
    },
    xaxis=dict(tickformat=',')  # Ensuring the price is not in scientific notation
)

# Show the figure without Plotly watermark
fig.show(config={'displaylogo': False})

### AVG Price Distribution for New and Used Non-Luxury Brands

In [48]:

# Create subplots with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, subplot_titles=('AVG New Non-Luxury Price Distribution', 'AVG Used Non-Luxury Price Distribution'))

# Plot AVG New Non-Luxury
fig.add_trace(
    go.Box(x=new_nonlux['price'], name=''),
    row=1, col=1
)

# Plot AVG Used Non-Luxury
fig.add_trace(
    go.Box(x=used_nonlux['price'], name=''),
    row=1, col=2
)

# Update layout for the entire figure
fig.update_layout(
    xaxis_title='Price',
    height=800,
    width=1150,
    title={
        #'text': 'Combined Non-Luxury Price Distributions',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 18, 'family': 'bold'}
    },
    xaxis=dict(tickformat=',')  # Ensuring the price is not in scientific notation
)

# Show the figure without Plotly watermark
fig.show(config={'displaylogo': False})

### Total AVG Price Distribution per New and Used Luxury and Non-Luxury Brands
- Luxury
    - AVG New Luxury Price Distribution: Prices mostly range between $50,000 and $150,000, with some outliers above $200,000.
    - AVG Used Luxury Price Distribution: Prices vary widely, with most between $0 and $100,000, but some outliers up to $1.5 million.
- Non-Luxury
    - AVG New Non-Luxury Price Distribution: Prices typically range between $20,000 and $60,000.
    - AVG Used Non-Luxury Price Distribution: Prices are mostly between $0 and $50,000, with some outliers up to $150,000.

#### Insights:
-  Luxury
    - New Luxury Cars: Have a higher price range compared to new non-luxury cars.
    - Used Luxury Cars: Show a wide range of prices, indicating variability in the condition and model of the cars.
-  Non-Luxury
    - New Non-Luxury Cars: Have a more consistent price range.
    - Used Non-Luxury Cars: Generally cheaper, but some outliers indicate high-value used cars.

#### Actionable Insight:
- Buyers: If you are looking for a new car, expect to pay significantly more for luxury brands. Used luxury cars can vary greatly in price, so shop around for the best deals.
- Sellers: Highlight the unique features and conditions of high-priced used luxury cars to justify their cost.

## Average Price per City

### AVG New Luxury and Non-Luxury Pricing per City

In [49]:
# Convert price to integer
avg_lux_price_per_city['price'] = avg_lux_price_per_city['price'].astype(int)
avg_nonlux_price_per_city['price'] = avg_nonlux_price_per_city['price'].astype(int)

# Sort values by brand (if not already sorted)
avg_lux_price_per_city = avg_lux_price_per_city.sort_values(by='brand', ascending=True)
avg_nonlux_price_per_city = avg_nonlux_price_per_city.sort_values(by='brand', ascending=True)

# Filter by status: new
filtered_lux_data = avg_lux_price_per_city[avg_lux_price_per_city['status'] == 'New']
filtered_nonlux_data = avg_nonlux_price_per_city[avg_nonlux_price_per_city['status'] == 'New']

# Create subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=('AVG Luxury Pricing per City (New)', 'AVG Non-Luxury Pricing per City (New)'))

# Add Luxury scatter plot to subplot
for city in filtered_lux_data['city'].unique():
    city_data = filtered_lux_data[filtered_lux_data['city'] == city]
    fig.add_trace(
        go.Scatter(
            x=city_data['brand'],
            y=city_data['price'],
            mode='markers',
            marker=dict(size=12),
            name=city,
            legendgroup=city
        ),
        row=1, col=1
    )

# Add Non-Luxury scatter plot to subplot
for city in filtered_nonlux_data['city'].unique():
    city_data = filtered_nonlux_data[filtered_nonlux_data['city'] == city]
    fig.add_trace(
        go.Scatter(
            x=city_data['brand'],
            y=city_data['price'],
            mode='markers',
            marker=dict(size=12),
            name=city,
            legendgroup=city,
            showlegend=False
        ),
        row=1, col=2
    )

# Update layout for both plots
fig.update_layout(
    title='AVG Pricing per City (New)',
    height=800,
    width=1150,
    font=dict(size=16),
    showlegend=True,
    legend=dict(x=1.02, y=1),
    margin=dict(l=60, r=60, t=80, b=30)
)

# Update axes for Luxury plot
fig.update_xaxes(title_text='Luxury Models', tickangle=-60, row=1, col=1)
fig.update_yaxes(title_text='Price', row=1, col=1)

# Update axes for Non-Luxury plot
fig.update_xaxes(title_text='Non-Luxury Models', tickangle=-60, row=1, col=2)
fig.update_yaxes(
    title_text='Price',
    type='log',
    tickvals=[1000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000],
    ticktext=["$1k", "$20k", "$30k", "$40K", "$50K", "$60K", "$70k", "$80k", "$90k", "$100k", "$110k", "$120k"],
    row=1, col=2
)

# Show the figure without Plotly watermark
fig.show(config={'displaylogo': False})

## AVG Used Luxury and Non-Luxury Pricing per City

In [50]:
#### AVG Used Pricing per City
# Convert price to integer
avg_lux_price_per_city['price'] = avg_lux_price_per_city['price'].astype(int)
avg_nonlux_price_per_city['price'] = avg_nonlux_price_per_city['price'].astype(int)

# Sort values by brand (if not already sorted)
avg_lux_price_per_city = avg_lux_price_per_city.sort_values(by='brand', ascending=True)
avg_nonlux_price_per_city = avg_nonlux_price_per_city.sort_values(by='brand', ascending=True)

# Filter by status: used and new
filtered_lux_data_used = avg_lux_price_per_city[avg_lux_price_per_city['status'] == 'Used']
filtered_nonlux_data_new = avg_nonlux_price_per_city[avg_nonlux_price_per_city['status'] == 'New']

# Create subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=('AVG Luxury Pricing per City (Used)', 'AVG Non-Luxury Pricing per City (Used)'))

# Add Luxury scatter plot to subplot
for city in filtered_lux_data_used['city'].unique():
    city_data = filtered_lux_data_used[filtered_lux_data_used['city'] == city]
    fig.add_trace(
        go.Scatter(
            x=city_data['brand'],
            y=city_data['price'],
            mode='markers',
            marker=dict(size=12),
            name=city,
            legendgroup=city
        ),
        row=1, col=1
    )

# Add Non-Luxury scatter plot to subplot
for city in filtered_nonlux_data_new['city'].unique():
    city_data = filtered_nonlux_data_new[filtered_nonlux_data_new['city'] == city]
    fig.add_trace(
        go.Scatter(
            x=city_data['brand'],
            y=city_data['price'],
            mode='markers',
            marker=dict(size=12),
            name=city,
            legendgroup=city,
            showlegend=False  # Only show legend for the first plot
        ),
        row=1, col=2
    )

# Update layout for both plots
fig.update_layout(
    title='AVG Pricing per City (Used)',
    height=800,
    width=1150,
    font=dict(size=16),
    showlegend=True,
    legend=dict(x=1.02, y=1),
    margin=dict(l=60, r=60, t=80, b=30)
)

# Update axes for Luxury plot
fig.update_xaxes(title_text='Luxury Models', tickangle=-60, categoryorder='category ascending', row=1, col=1)
fig.update_yaxes(title_text='Price', type='log', tickvals=[1, 10, 100, 1000, 10000, 100000, 1000000], ticktext=["$1", "$10", "$100", "$1K", "$10K", "$100K", "$1M"], row=1, col=1)

# Update axes for Non-Luxury plot
fig.update_xaxes(title_text='Non-Luxury Models', tickangle=-60, categoryorder='category ascending', row=1, col=2)
fig.update_yaxes(
    title_text='Price',
    type='log',
    tickvals=[1000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000],
    ticktext=["$1k", "$20k", "$30k", "$40K", "$50K", "$60K", "$70k", "$80k", "$90k", "$100k", "$110k", "$120k"],
    row=1, col=2
)

# Show plotly figure
fig.show()

### City Comparisons
- **Atlanta**: High-end luxury brands dominate, with Aston Martin and Ferrari leading in price.
- **Chicago**: Audi and Ferrari are top high-priced brands.
- **Columbus**: Mercedes-Benz and Ferrari are prevalent in both new and used markets.

#### Key Insights:
- **New Cars**: High-end brands like Aston Martin and Mercedes-Benz are prevalent in major cities.
- **Used Cars**: Ferrari and Rolls-Royce maintain high prices, indicating strong market demand.

#### Actionable Insights:
- **Buyers**: Consider cities like Atlanta and Chicago for a wide range of high-end luxury cars.
- **Sellers**: Emphasize the exclusivity and high value of luxury brands to attract buyers.

## AVG New and Used Luxry Brands Offered per City

In [51]:
# seller_new_lux =  new_lux[['seller_name', 'city', 'year', 'brand', 'model', 'price']].copy()
# seller_used_lux = used_lux[['seller_name', 'city', 'year', 'brand', 'model', 'price']].copy()

# seller_new_nonlux =  new_nonlux[['seller_name', 'city', 'year', 'brand', 'model', 'price']].copy()
# seller_used_nonlux = used_nonlux[['seller_name', 'city', 'year', 'brand', 'model', 'price']].copy()

# # Combine all data into one DataFrame with an additional 'type' column
# seller_new_lux['type'] = 'New Luxury'
# seller_used_lux['type'] = 'Used Luxury'
# seller_new_nonlux['type'] = 'New Non-Luxury'
# seller_used_nonlux['type'] = 'Used Non-Luxury'

# combined_data = pd.concat([seller_new_lux, seller_used_lux, seller_new_nonlux, seller_used_nonlux])

# # Create a list of unique cities and brands
# cities = combined_data['city'].unique()
# brands = combined_data['brand'].unique()

# # Determine the number of columns for subplots
# cols = 3

# # Create separate figures for each city
# for city in cities:
#     # Filter data for the current city
#     city_data = combined_data[combined_data['city'] == city]
    
#     # Filter out brands with no data in the current city
#     available_brands = city_data['brand'].unique()
    
#     # Determine the number of rows for subplots
#     num_brands = len(available_brands)
#     rows = (num_brands // cols) + 1 if num_brands % cols != 0 else num_brands // cols
    
#     # Create subplots for each brand within the city
#     fig = make_subplots(rows=rows, cols=cols, subplot_titles=available_brands)
    
#     # Add bar traces to the subplots
#     for j, brand in enumerate(available_brands):
#         brand_data = city_data[city_data['brand'] == brand]
#         if not brand_data.empty:
#             row = (j // cols) + 1
#             col = (j % cols) + 1
#             fig.add_trace(
#                 go.Bar(
#                     x=brand_data['seller_name'],
#                     y=brand_data['price'],
#                     name=brand,
#                     marker_color=city_colors.get(city, 'gray')
#                 ),
#                 row=row, col=col
#             )
    
#     # Update layout
#     fig.update_layout(
#         title_text=f'Seller Prices in {city} by Brand',
#         height=400 * rows,
#         width=1000,
#         showlegend=False  # Hide legend to avoid repetition
#     )
    
#     # Update x-axis tick rotation for better readability
#     for j in range(num_brands):
#         row = (j // cols) + 1
#         col = (j % cols) + 1
#         fig.update_xaxes(tickangle=-45, row=row, col=col)
    
#     # Show plotly figure
#     fig.show(config={'displaylogo': False})

In [52]:
# Number of New Lux brands offered per city
new_lux_city_brands_offered = new_lux.groupby('city')['brand'].nunique().reset_index()
new_lux_city_brands_offered.columns = ['city', 'count']

# Number of Used Lux brands offered per city
used_lux_city_brands_offered = used_lux.groupby('city')['brand'].nunique().reset_index()
used_lux_city_brands_offered.columns = ['city', 'count']

# Create subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=('Total New Luxury Brands Offered per City', 'Total Used Luxury Brands Offered per City'))

# Add New Luxury Brands plot to subplot
fig.add_trace(
    go.Bar(
        x=new_lux_city_brands_offered['city'],
        y=new_lux_city_brands_offered['count'],
        name='New Luxury',
        marker_color=[city_colors[city] for city in new_lux_city_brands_offered['city']]
    ),
    row=1, col=1
)

# Add Used Luxury Brands plot to subplot
fig.add_trace(
    go.Bar(
        x=used_lux_city_brands_offered['city'],
        y=used_lux_city_brands_offered['count'],
        name='Used Luxury',
        marker_color=[city_colors[city] for city in used_lux_city_brands_offered['city']]
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text='Luxury Brands Offered per City',
    height=600,
    width=1150,
    showlegend=False  # Hide legend to avoid repetition
)

# Update x-axis tick rotation for better readability
fig.update_xaxes(tickangle=-45, row=1, col=1)
fig.update_xaxes(tickangle=-45, row=1, col=2)

# Show plotly figure
fig.show()

## Non-Luxury Brands Offered

In [53]:
# Number of New Lux brands offered per city
new_nonlux_city_brands_offered = new_nonlux.groupby('city')['brand'].nunique().reset_index()
new_nonlux_city_brands_offered.columns = ['city', 'count']

# Number of Used Lux brands offered per city
used_nonlux_city_brands_offered = used_nonlux.groupby('city')['brand'].nunique().reset_index()
used_nonlux_city_brands_offered.columns = ['city', 'count']

# Create subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=('Total New Non-Luxury Brands Offered per City', 'Total Used Non-Luxury Brands Offered per City'))

# Add New Luxury Brands plot to subplot
fig.add_trace(
    go.Bar(
        x=new_nonlux_city_brands_offered['city'],
        y=new_nonlux_city_brands_offered['count'],
        name='New Non-Luxury',
        marker_color=[city_colors[city] for city in new_nonlux_city_brands_offered['city']]
    ),
    row=1, col=1
)

# Add Used Luxury Brands plot to subplot
fig.add_trace(
    go.Bar(
        x=used_nonlux_city_brands_offered['city'],
        y=used_nonlux_city_brands_offered['count'],
        name='Used Non-Luxury',
        marker_color=[city_colors[city] for city in used_nonlux_city_brands_offered['city']]
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text='Non-Luxury Brands Offered per City',
    height=600,
    width=1150,
    showlegend=False  # Hide legend to avoid repetition
)

# Update x-axis tick rotation for better readability
fig.update_xaxes(tickangle=-45, row=1, col=1)
fig.update_xaxes(tickangle=-45, row=1, col=2)

# Show plotly figure
fig.show()

## Non-Luxury Brands Offered per City
#### Total New Luxury Brands Offered per City:
- Highest in Atlanta and Houson (6 brands).
- Cities like Columbus, Denver, and Seattle offer around 5 brands.
- New York, Philadelphia, Pheonix, and San Diego offer fewer brands (1-3 brands).

#### Total Used Luxury Brands Offered per City:
- Highest in Dallas (21 brands).
- Cities like Houston and Philadelphia offer 18 brands.
- Denver and Colombus offer fewer brands (around 12-13 brands).

#### Insights:
- New Luxury Cars: Atlanta and Houson has the widest variety of new luxury brands.
- Used Luxury Cars: Dallas, Houston, and Philadelphia offers the most variety in used luxury brands.

#### Actionable Insight:
- Buyers: For a wide selection of new luxury cars, consider shopping in Atlanta and Houson. For used luxury cars, Dallas offers the most options.
- Sellers: In cities with fewer brands, highlight the exclusivity and premium nature of your offerings to attract buyers.

#### Total New Non-Luxury Brands Offered per City:
- Highest in Columbus (9 brands).
- Cities like Denver and Atlanta offer around 7-8 brands.
- Chicago, Dallas, New York, and Pheonix offer fewer brands (around 4 brands).


#### Total Used Non-Luxury Brands Offered per City:
- Highest in Atlanta and New York (22 brands).
- Cities like Dallas, Denver, San Diego, and Seattle offer 20 brands.
- Los Angeles and Columbus offer fewer brands (around 15-16 brands).

#### Insights:
- New Luxury Cars: Columbus has the widest variety of new luxury brands.
- Used Luxury Cars: Atlanta and New York offer the most variety in used luxury brands.

#### Actionable Insight:
- Buyers: For a wide selection of new luxury cars, consider shopping in Columbus. For used luxury cars, Atlanta and New York offer the most options.
- Sellers: In cities with fewer brands, highlight the exclusivity and premium nature of your offerings to attract buyers.




## Top Brands

In [54]:
# Top New Luxury brands
top_brands_counts_new = new_lux['brand'].value_counts().reset_index()
top_brands_counts_new.columns = ['brand', 'count']

# Top Used Luxury brands
top_brands_counts_used = used_lux['brand'].value_counts().reset_index()
top_brands_counts_used.columns = ['brand', 'count']

# Create subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=('Top New Luxury Brands', 'Top Used Luxury Brands'))

# Add New Luxury bar plot to subplot
fig.add_trace(
    go.Bar(
        x=top_brands_counts_new['brand'],
        y=top_brands_counts_new['count'],
        name='New Luxury',
        marker=dict(color='blue')
    ),
    row=1, col=1
)

# Add Used Luxury bar plot to subplot
fig.add_trace(
    go.Bar(
        x=top_brands_counts_used['brand'],
        y=top_brands_counts_used['count'],
        name='Used Luxury',
        marker=dict(color='orange')
    ),
    row=1, col=2
)

# Update layout for both plots
fig.update_layout(
    title='Top Luxury Brands (New vs Used)',
    height=500,
    width=1150,
    font=dict(size=16),
    showlegend=True,
    legend=dict(x=1.02, y=1),
    margin=dict(l=60, r=60, t=80, b=30)
)

# Update axes for New Luxury plot
fig.update_xaxes(title_text='Brand', tickangle=-80, row=1, col=1)
fig.update_yaxes(title_text='Count', row=1, col=1)

# Update axes for Used Luxury plot
fig.update_xaxes(title_text='Brand', tickangle=-80, row=1, col=2)
fig.update_yaxes(title_text='Count', row=1, col=2)

# Show plotly figure
fig.show()
#### Top Non-Luxury Brands
# Top New Non-Luxury brands
top_brands_counts_new_nonlux = new_nonlux['brand'].value_counts().reset_index()
top_brands_counts_new_nonlux.columns = ['brand', 'count']

# Top Used Non-Luxury brands
top_brands_counts_used_nonlux = used_nonlux['brand'].value_counts().reset_index()
top_brands_counts_used_nonlux.columns = ['brand', 'count']

# Create subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=('Top New Non-Luxury Brands', 'Top Used Non-Luxury Brands'))

# Add New Non-Luxury bar plot to subplot
fig.add_trace(
    go.Bar(
        x=top_brands_counts_new_nonlux['brand'],
        y=top_brands_counts_new_nonlux['count'],
        name='New Non-Luxury',
        marker=dict(color='green')
    ),
    row=1, col=1
)

# Add Used Non-Luxury bar plot to subplot
fig.add_trace(
    go.Bar(
        x=top_brands_counts_used_nonlux['brand'],
        y=top_brands_counts_used_nonlux['count'],
        name='Used Non-Luxury',
        marker=dict(color='red')
    ),
    row=1, col=2
)

# Update layout for both plots
fig.update_layout(
    title='Top Non-Luxury Brands (New vs Used)',
    height=500,
    width=1150,
    font=dict(size=16),
    showlegend=True,
    legend=dict(x=1.02, y=1),
    margin=dict(l=60, r=60, t=80, b=30)
)

# Update axes for New Non-Luxury plot
fig.update_xaxes(title_text='Brand', tickangle=-80, row=1, col=1)
fig.update_yaxes(title_text='Count', row=1, col=1)

# Update axes for Used Non-Luxury plot
fig.update_xaxes(title_text='Brand', tickangle=-80, row=1, col=2)
fig.update_yaxes(title_text='Count', row=1, col=2)

# Show plotly figure
fig.show()

## Top Luxury Brands (New vs Used)
#### Top New Luxury Brands:Most popular brands: Mercedes-Benz, BMW, Audi, and Polestar.
- Less popular brands: McLaren, INFINITI, Land Rover, and Maserati.
#### Top Used Luxury Brands:
- Most popular brands: BMW, Mercedes-Benz, Cadillac, and Ferrari.
- Less popular brands: Rivian, Maybach, Alfa, and Genesis.
#### Insights:
- New Luxury Cars: Mercedes-Benz and BMW are the most offered brands.
- Used Luxury Cars: BMW and Mercedes-Benz are the most offered brands.
#### Actionable Insight:
- Buyers: For new luxury cars, Mercedes-Benz and BMW are widely available. For used luxury cars, BMW and Mercedes-Benz have the most options.
- Sellers: Emphasize the high demand for top brands like BMW and Mercedes-Benz to attract more buyers.

## Top Non-Luxury Brands (New vs Used)
#### Top New Non-Luxury Brands:
- Most popular brands: RAM, Hyundai, Jeep, Ford, and Honda.
- Less popular brands: Subaru, GMC, MINI, and Mazda.
#### Top Used Non-Luxury Brands:
- Most popular brands: Toyota, Nissan, Hyundai, Dodge, and Honda.
- Less popular brands: Isuzu, Plymouth, Oldsmobile, and Saturn.
#### Insights:
- New Non-Luxury Cars: RAM and Hyundai are the most offered brands.
- Used Non-Luxury Cars: Toyota and Nissan are the most offered brands.
#### Actionable Insight:
- Buyers: If you are looking for a new non-luxury car, RAM and Hyundai are widely available. For used non-luxury cars, Toyota and Nissan have the most options.
- Sellers: Highlight the popularity of top brands like RAM and Toyota to attract more buyers.


## Comparing Average Pricing per Seller

In [55]:
# Combine data into one DataFrame
seller_new_lux = new_lux[['seller_name', 'city', 'year', 'brand', 'model', 'price']].copy()
seller_used_lux = used_lux[['seller_name', 'city', 'year', 'brand', 'model', 'price']].copy()
seller_new_nonlux = new_nonlux[['seller_name', 'city', 'year', 'brand', 'model', 'price']].copy()
seller_used_nonlux = used_nonlux[['seller_name', 'city', 'year', 'brand', 'model', 'price']].copy()

# Add 'type' column
seller_new_lux['type'] = 'New Luxury'
seller_used_lux['type'] = 'Used Luxury'
seller_new_nonlux['type'] = 'New Non-Luxury'
seller_used_nonlux['type'] = 'Used Non-Luxury'

# Combine into one DataFrame
combined_data = pd.concat([seller_new_lux, seller_used_lux, seller_new_nonlux, seller_used_nonlux])

# Calculate mean prices by grouping
mean_prices = combined_data.groupby(['city', 'seller_name', 'brand', 'type'])['price'].mean().reset_index()
mean_prices['price'] = mean_prices['price'].astype(str).replace(r'\..*', '', regex=True)

# Apply mean prices to original DataFrame
combined_data = pd.merge(combined_data, mean_prices, on=['city', 'seller_name', 'brand', 'type'], suffixes=('', '_mean'))
combined_data.drop_duplicates(subset=['city', 'seller_name', 'brand', 'price_mean', 'type'] , inplace=True)


# Create a Plotly table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=["City", 'Seller Name', "Brand", "Price", "Type"],
        fill_color='paleturquoise',
        align='left'
    ),
    cells=dict(
        values=[
            combined_data['city'],
            combined_data['seller_name'],
            combined_data['brand'],
            combined_data['price_mean'],  # Use mean price here
            combined_data['type']
        ],
        fill_color='lavender',
        align='left'
    )
)])

# Update layout
fig.update_layout(
    title_text='Seller Prices by City, Seller, Brand, Price and Type',
    height=600,
    width=1200
)

# Show plotly table
fig.show(config={'displaylogo': False})

### Top 5 Sellers by Average Price
These sellers have the highest average prices for their listings, indicating they primarily deal in high-end or luxury vehicles:
- Earth MotorCars / Lotus of Dallas - Average Price: $259,900
- Ferrari Los Angeles - Average Price: $419,000
- Lamborghini Houston, Rolls-Royce North Houston - Average Price: $276,983
- Continental Ferrari (Chicago) - Average Price: $257,499
- McLaren Philadelphia - Average Price: $314,808
### Bottom 5 Sellers by Average Price
These sellers have the lowest average prices for their listings, indicating they primarily deal in more affordable, non-luxury vehicles:
- Auto's of Chicago - Average Price: $13,155
- Driven Auto of Oak Forest (Chicago) - Average Price: $13,480
- Affinity Auto Sales (Chicago) - Average Price: $12,995
- Auto House Oakbrook (Chicago) - Average Price: $8,490
- Auto House Motors (Chicago) - Average Price: $5,350
### Sellers with the Most Listings
These sellers have the highest number of listings, indicating a broad inventory and possibly a larger market presence:
- CarMax - Multiple locations across various cities
- Audi McKinney (Dallas)
- Mercedes-Benz of Houston Greenway
- BMW of Houston North
- DriveTime - Multiple locations across various cities
### Competitive Pricing Insights
- Luxury Vehicles:
    - High-End Models: Sellers like Ferrari Los Angeles and Lamborghini Houston maintain high average prices due to the exclusivity of their models.
    - Mid-Range Luxury: Sellers such as Audi McKinney and BMW of Houston North offer competitive pricing on popular luxury models, making them attractive options for buyers.
    - City Variations: Prices for the same models can vary significantly by city. For example, the Audi Q5 Sportback Premium is listed at $55,840 in Atlanta and $56,335 in Chicago.
- Non-Luxury Vehicles:
    - Affordable Options: Sellers like Auto's of Chicago and Driven Auto of Oak Forest offer very affordable prices, making them attractive for budget-conscious buyers.
    - City Comparisons: Buyers can find better deals in certain cities. For instance, a 2019 Nissan Rogue is more affordable in Phoenix ($17,695) compared to Los Angeles ($21,998).
### Business/Seller Insights
- High-End Sellers: Sellers like Earth MotorCars / Lotus of Dallas and Ferrari Los Angeles cater to a niche market with high-end luxury - vehicles, maintaining high average prices.
- Volume Sellers: CarMax and DriveTime, with multiple locations and a large number of listings, dominate the market by offering a wide range of vehicles at competitive prices.
- City-Specific Strategies: Sellers in cities like Chicago and Dallas offer a mix of high-end and affordable vehicles, catering to diverse customer bases.
### Buyer Insights
- Luxury Vehicle Buyers:
    - High-End Buyers: Should consider sellers like Ferrari Los Angeles and Lamborghini Houston for exclusive models.
    - Mid-Range Luxury Buyers: Should compare prices across cities and sellers like Audi McKinney and BMW of Houston North for competitive deals.
    - Certified Pre-Owned Options: Buyers can find certified pre-owned luxury vehicles at competitive prices from sellers like Audi McKinney.
- Non-Luxury Vehicle Buyers:
    - Budget-Conscious Buyers: Should look at sellers like Auto's of Chicago and Driven Auto of Oak Forest for the most affordable options.
    - City Comparisons: Buyers should compare prices across cities to find the best deals, especially in cities like Phoenix and Los Angeles.
### Competitive Conclusion
For sellers, understanding the competitive landscape and pricing strategies in different cities can help in setting attractive prices. For buyers, comparing prices across cities can lead to significant savings, especially for high-end luxury vehicles. Both sellers and buyers should leverage these insights to make informed decisions in the automotive market.


## Total Fuel Types
### Luxury Fuel Types

In [56]:
# Replace '–' with empty string in fuel_type column for luxury vehicles
new_lux['fuel_type'] = new_lux['fuel_type'].replace('–', '')
used_lux['fuel_type'] = used_lux['fuel_type'].replace('–', '')

# Filter out null values in fuel_type column for luxury vehicles
new_lux_fuel = new_lux[new_lux['fuel_type'] != '']
used_lux_fuel = used_lux[used_lux['fuel_type'] != '']

# Count of luxury fuel types after replacement and filtering
new_lux_fuel_types = new_lux_fuel['fuel_type'].value_counts()
used_lux_fuel_types = used_lux_fuel['fuel_type'].value_counts()

# Replace '–' with empty string in fuel_type column for non-luxury vehicles
new_nonlux['fuel_type'] = new_nonlux['fuel_type'].replace('–', '')
used_nonlux['fuel_type'] = used_nonlux['fuel_type'].replace('–', '')

# Filter out null values and 'Other' in fuel_type column for non-luxury vehicles
new_nonlux_fuel = new_nonlux[(new_nonlux['fuel_type'] != '') & (new_nonlux['fuel_type'] != 'Other')]
used_nonlux_fuel = used_nonlux[(used_nonlux['fuel_type'] != '') & (used_nonlux['fuel_type'] != 'Other')]

# Count of non-luxury fuel types after replacement and filtering
new_nonlux_fuel_types = new_nonlux_fuel['fuel_type'].value_counts()
used_nonlux_fuel_types = used_nonlux_fuel['fuel_type'].value_counts()

# Create subplots with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, subplot_titles=('Luxury Fuel Types Comparison', 'Non-Luxury Fuel Types Comparison'))

# Add bar trace for New Luxury Fuel Types
fig.add_trace(
    go.Bar(x=new_lux_fuel_types.index, y=new_lux_fuel_types.values, name='New Luxury'),
    row=1, col=1
)

# Add bar trace for Used Luxury Fuel Types
fig.add_trace(
    go.Bar(x=used_lux_fuel_types.index, y=used_lux_fuel_types.values, name='Used Luxury'),
    row=1, col=1
)

# Add bar trace for New Non-Luxury Fuel Types
fig.add_trace(
    go.Bar(x=new_nonlux_fuel_types.index, y=new_nonlux_fuel_types.values, name='New Non-Luxury'),
    row=1, col=2
)

# Add bar trace for Used Non-Luxury Fuel Types
fig.add_trace(
    go.Bar(x=used_nonlux_fuel_types.index, y=used_nonlux_fuel_types.values, name='Used Non-Luxury'),
    row=1, col=2
)

# Update layout for the entire figure
fig.update_layout(
    height=600,
    width=1150,
    showlegend=True,
    legend=dict(x=0.01, y=0.99),  # Adjust legend position
    title_text='Fuel Types Comparison',
    title_x=0.5,  # Centered title
    xaxis_title='Fuel Type',
    yaxis_title='Count',
    barmode='group',  # Group bars for comparison
)

# Show the combined plot
fig.show()

### Total Fuel Types
#### Luxury Fuel Types Comparison:
- Most common fuel type: Gasoline.
- Less common fuel types: Hybrid, Electric, and Diesel.
#### Non-Luxury Fuel Types Comparison:
- Most common fuel type: Gasoline.
- Less common fuel types: Hybrid and Diesel.
#### Insights:
- Luxury Cars: Gasoline is the predominant fuel type for both new and used luxury cars.
- Non-Luxury Cars: Gasoline is also the predominant fuel type, but there is a notable presence of hybrid vehicles.
#### Actionable Insight:
- Buyers: If you prefer gasoline cars, you will have many options in both luxury and non-luxury segments. For eco-friendly options, consider hybrid cars, especially in the non-luxury segment.
- Sellers: Highlight the fuel efficiency and environmental benefits of hybrid cars to attract eco-conscious buyers.


## Average Mileage

### AVG Luxury Mileage

In [57]:
# Average luxury mileage
new_lux_avg_mileage = new_lux['mileage'].mean().astype(int)
used_lux_avg_mileage = used_lux['mileage'].mean().astype(int)
print('Average New Luxury Mileage:', new_lux_avg_mileage, 'Average Used Luxury Mileage:', used_lux_avg_mileage)

Average New Luxury Mileage: 113 Average Used Luxury Mileage: 56585


### AVG Non-Luxury Mileage

In [58]:
# Average non-luxury mileage
new_nonlux_avg_mileage = new_nonlux['mileage'].mean().astype(int)
used_nonlux_avg_mileage = used_nonlux['mileage'].mean().astype(int)
print('Average New Non-Luxury Mileage:', new_nonlux_avg_mileage, 'Average Used Non-Luxury Mileage:', used_nonlux_avg_mileage)

Average New Non-Luxury Mileage: 55 Average Used Non-Luxury Mileage: 76800


### Average Mileage


#### Luxury
- New Luxury Cars: Average mileage is 111 miles.
- Used Luxury Cars: Average mileage is 56,522 miles.
#### Non-Luxury
- Average New Non-Luxury Mileage: 54 miles
- Average Used Non-Luxury Mileage: 76,800 miles
#### Insights:
- Luxury:
    - New luxury cars typically have very low mileage, indicating they are mostly brand new and have been driven minimally, likely for test drives or transportation purposes.
    - Used luxury cars have moderate mileage, suggesting they have been driven for a few years. This mileage is relatively lower compared to used non-luxury cars, which might indicate better maintenance and less frequent use.
- Non-Luxury
    - New non-luxury cars also have very low mileage, similar to new luxury cars. This low mileage confirms that these vehicles are fresh from the factory and have seen minimal use.
    - Used non-luxury cars have higher mileage compared to used luxury cars. This higher mileage suggests that non-luxury cars are driven more extensively, possibly due to their use as daily drivers or for longer commutes.
#### Actionable Insight:
- For Buyers:
    - New Cars: Both luxury and non-luxury new cars have very low mileage, making them a good choice for those looking for a brand-new vehicle with minimal wear and tear.
    - Used Cars: If considering used cars, luxury cars might be a better option for those looking for lower mileage and potentially better maintenance. However, used non-luxury cars might offer more affordable options but with higher mileage.
- For Sellers:
    - New Cars: Emphasize the low mileage of new cars to attract buyers looking for nearly untouched vehicles.
    - Used Cars: Highlight the relatively lower mileage of used luxury cars to justify higher prices and attract buyers looking for well-maintained vehicles. For used non-luxury cars, focus on affordability and the value they offer despite higher mileage.

## At least 1 accident reported

### Luxury Accidents Reported

In [59]:
# Count of luxury accidents reported
new_lux_accidents = new_lux['accidents_or_damage'].value_counts()
used_lux_accidents = used_lux['accidents_or_damage'].value_counts()
print('New Luxury Accidents Reported:', new_lux_accidents, 'Used Luxury Accidents Reported:', used_lux_accidents)

New Luxury Accidents Reported: Series([], Name: count, dtype: int64) Used Luxury Accidents Reported: accidents_or_damage
None reported                             591
At least 1 accident or damage reported    216
Name: count, dtype: int64


### Non-Luxury Accidents Reported

In [60]:
# Count of non-luxury accidents reported
new_nonlux_accidents = new_nonlux['accidents_or_damage'].value_counts()
used_nonlux_accidents = used_nonlux['accidents_or_damage'].value_counts()
print('New Non-Luxury Accidents Reported:', new_nonlux_accidents, 'Used Non-Luxury Accidents Reported:', used_nonlux_accidents)

New Non-Luxury Accidents Reported: Series([], Name: count, dtype: int64) Used Non-Luxury Accidents Reported: accidents_or_damage
None reported                             545
At least 1 accident or damage reported    271
Name: count, dtype: int64


### Accident Reports
- Luxury"
    - New Luxury Cars: No accident data reported.
    - Used Luxury Cars:
        - 592 cars with no accidents.
        - 216 cars with at least one accident.
    - New Non-Luxury Cars: No accident data reported.
    - Used Non-Luxury Cars:
        - 545 cars with no accidents.
        - 271 cars with at least one accident.
#### Insights:
- New Cars: No accidents reported, which is typical for new cars.
- Used Cars: A significant number of both luxury and non-luxury used cars have been in accidents. However, more used non-luxury cars have been in accidents compared to luxury cars.
#### Actionable Insight:
- When buying a used car, check the accident history. Used non-luxury cars have a slightly higher chance of having been in an accident compared to used luxury cars. This could impact the car's safety and resale value.


## Key Findings
### Average Pricing
#### Luxury Cars:
- New Luxury Cars: Prices mostly range between $50,000 and $150,000, with some outliers above $200,000.
- Used Luxury Cars: Prices vary widely, with most between $0 and $100,000, but some outliers up to $1.5 million.
#### Non-Luxury Cars:
- New Non-Luxury Cars: Prices typically range between $20,000 and $60,000.
- Used Non-Luxury Cars: Prices are mostly between $0 and $50,000, with some outliers up to $150,000.
### Brand Popularity
#### Luxury Brands:
- New: Mercedes-Benz, BMW, Audi, and Polestar are the most popular.
- Used: BMW, Mercedes-Benz, Cadillac, and Ferrari dominate the market.
#### Non-Luxury Brands:
- New: RAM, Hyundai, Jeep, Ford, and Honda are the top brands.
- Used: Toyota, Nissan, Hyundai, Dodge, and Honda are the most prevalent.
### Fuel Types
- Luxury Cars: Predominantly gasoline, with some hybrid and electric options.
- Non-Luxury Cars: Mostly gasoline, but a notable presence of hybrid vehicles.
### Mileage
#### Luxury Cars:
- New: Very low mileage, indicating they are mostly brand new.
- Used: Moderate mileage, suggesting they have been driven for a few years.
#### Non-Luxury Cars:
- New: Very low mileage, similar to new luxury cars.
- Used: Higher mileage compared to used luxury cars, indicating extensive use.
### Seller Insights
- High-End Sellers: Earth MotorCars / Lotus of Dallas and Ferrari Los Angeles cater to a niche market with high-end luxury vehicles.
- Volume Sellers: CarMax and DriveTime dominate the market with a wide range of vehicles at competitive prices.
### City-Specific Insights
- Atlanta: Offers a wide range of high-end luxury cars, with brands like Aston Martin and Ferrari dominating the market.
- Chicago: Features high-end new and used cars, particularly Audi and Ferrari.
- Columbus: Mercedes-Benz and Ferrari are among the top high-priced cars.
- Dallas: Offers a mix of high-end new and used cars, particularly Mercedes-Benz and Lamborghini.
- Denver: BMW and Ferrari are among the top high-priced cars.
- Houston: McLaren and Ferrari dominate the high-end market.
- Los Angeles: Mercedes-Benz and Bugatti are the most expensive cars available.
- New York: Volvo and Ferrari are among the top high-priced cars.
- Philadelphia: BMW and Ferrari dominate the high-end market.
- Phoenix: Audi and Ferrari are the most expensive cars available.
- San Diego: BMW and Ferrari are among the top high-priced cars.
- Seattle: Mercedes-Benz and Lamborghini dominate the high-end market.
### Final Recommendations
#### For Buyers:
- Luxury Car Buyers:
    - High-end buyers should consider sellers like Ferrari Los Angeles and Lamborghini Houston for exclusive models.
    - Mid-range luxury buyers should compare prices across cities for competitive deals.
    - Certified pre-owned options are available at competitive prices from sellers like Audi McKinney.
- Non-Luxury Car Buyers:
    - Budget-conscious buyers should look at sellers like Auto's of Chicago for the most affordable options.
    - Compare prices across cities to find the best deals, especially in cities like Phoenix and Los Angeles.
#### For Sellers:
- Luxury Car Sellers:
    - Emphasize the exclusivity and high value of luxury brands to attract buyers.
    - Highlight the new model year and high ratings to attract buyers looking for reliable and high-quality purchases.
- Non-Luxury Car Sellers:
    - Highlight the fuel efficiency and environmental benefits of hybrid cars to attract eco-conscious buyers.
    - Emphasize the low mileage of new cars to attract buyers looking for nearly untouched vehicles.
    - For used cars, focus on affordability and the value they offer despite higher mileage.
### Conclusion
This analysis provides valuable insights into the automotive market, helping both buyers and sellers make informed decisions. By understanding pricing trends, brand popularity, fuel types, and seller performance, stakeholders can optimize their strategies to achieve better outcomes in the market.